<a href="https://colab.research.google.com/github/jlandesman/nyc_covid_data/blob/master/Covid_NYC_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Not an Epidemiologist's Guide to NY/NYC Covid Cases

I have been frustrated by the graphs and presentations available on NY and NYC's progress in fighting Covid-19. 

The below graphs are purely for guiding my own assessment of how New York is doing, and are definitely not medical advice.

## About the Data

As the saying goes, "don't get into a fight trying to clean data.  You both get dirty, but the data likes it."  

New York City Covid data is directionally consistent across sources, but there appear to be large discrepancies between city and state.  sites like [The City](https://projects.thecity.nyc/2020_03_covid-19-tracker/) draw from the NYC Health Department.

Based on watching the data during the pandemic, I have decided to go with the State's [data](https://data.ny.gov/browse?tags=covid-19) for the overall data as it is based on hospital surveys. The NYC data, particularly the deaths data, is lagged as the NYC Health Dept. recieves its data from places like the city morgue, which has some processing backlogs. 

Note that the state updates their data ta **2 PM** each day

In [1]:
%precision %.2f
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
 
df              = pd.read_csv('https://health.data.ny.gov/api/views/xdss-u53e/rows.csv?accessType=DOWNLOAD')
df['Test Date'] = pd.to_datetime(df['Test Date'])
START_DATE      = '06/01/2020' ## Date for starting the graphs below

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
## Define Regions
nyc_counties = ['Kings','Queens','Richmond','New York','Bronx']
suburbs_counties = ['Westchester','Nassau','Suffolk', 'Rockland']
 
## Subset regions
nyc = df[df['County'].isin(nyc_counties)].groupby('Test Date').sum()
suburbs = df[df['County'].isin(suburbs_counties)].groupby('Test Date').sum()
upstate = df[(~df['County'].isin(nyc_counties)) & (~df['County'].isin(suburbs_counties))].groupby('Test Date').sum()
 
nyc['location'] = 'nyc'
suburbs['location'] = 'suburbs'
upstate['location'] = 'upstate'

## Concat and filter
daily_data = pd.concat([nyc,suburbs,upstate])
daily_data.reset_index(inplace=True)
daily_data.set_index('Test Date',inplace=True)
daily_data =daily_data.drop(['Cumulative Number of Positives', 'Cumulative Number of Tests Performed'], axis=1)
daily_data.columns = ['positives','tests','location']

## New Columns for  Analysis:
 - % Positive
 - Rolling 7 day averages
 - 1 day pct change
 - 7 day pct change

In [3]:
daily_data['pct_positive'] = daily_data['positives']/ daily_data['tests']*100
daily_data['pct_change_1_day'] = daily_data.groupby('location')['positives'].pct_change() * 100
daily_data['pct_change_7_day'] = daily_data.groupby('location')['positives'].pct_change(7) * 100
daily_data = daily_data[START_DATE:]

weekly_avgs = daily_data.groupby('location').rolling(7).mean()

## Latest Data

In [4]:
pd.set_option('precision', 2)
daily_data.pivot(columns='location',values=['positives','pct_positive']).tail(7)

positives                 pct_positive                
location         nyc suburbs upstate          nyc suburbs upstate
Test Date                                                        
2020-08-01     263.0   122.0   146.0         1.02    0.91    0.74
2020-08-02     241.0   127.0   177.0         0.97    1.10    1.16
2020-08-03     316.0   172.0   258.0         1.05    1.11    1.02
2020-08-04     301.0   167.0   168.0         1.02    1.03    0.62
2020-08-05     333.0   176.0   194.0         1.06    1.01    0.83
2020-08-06     344.0   133.0   237.0         1.13    0.88    0.97
2020-08-07     326.0   171.0   206.0         0.99    0.96    0.86

## New Positives since June 1

At this stage of the epidemic, I am mostly interested in three indicators: new cases, percent positive, and R_0.  I am under the assumption that as long as testing continues at high levels (50-70k/ day), then we are accurately capturing the majority of the cases. 

In [5]:
px.line(daily_data.reset_index(), 
        x='Test Date', 
        y='positives', 
        color='location',
        title = 'New Positives',
        template = 'simple_white',
        range_y =(0,700))

In [6]:
px.line(weekly_avgs.reset_index().dropna(), 
        x='Test Date', 
        y='positives', 
        color='location',
        title = 'New Positives, 7 Day Rolling Average',
        template = 'simple_white')

## Percent of Tests Returning positive

In [7]:
px.line(daily_data.reset_index(), 
        x='Test Date', 
        y='pct_positive', 
        color='location',
        title = 'Percent of Tests with Positive Results',
        template = 'simple_white',
        range_y =(0,2.5))

In [8]:
px.line(weekly_avgs.reset_index().dropna(), 
        x='Test Date', 
        y='pct_positive', 
        color='location',
        title = 'Percent Positive, 7 Day Rolling Average',
        template = 'simple_white',
        range_y =(0,2.5))

## R_0 estimate from Rt.Live

**To do** add in the error bars from rt.live (its in the dataframe above)

In [9]:
rt = pd.read_csv('https://d14wlfuexuxgcm.cloudfront.net/covid/rt.csv')
rt['date'] = pd.to_datetime(rt['date'])
rt.set_index('date', inplace=True)
rt = rt[START_DATE:]

In [10]:
px.line(rt[rt['region'] == 'NY'].reset_index(), 
        x='date', 
        y='mean', 
        title = 'R_0 Estimate for NY',
        template = 'simple_white')

## Hospitalizations in NYC

Note that this data is from the NYC Department of Health.  This is recorded as "by date of admission", and is lagged and subsequently revised.   Recent data is likely incomplete.  

This data is unclear if it is net admissions.  It appears to be not according to https://github.com/nychealth/coronavirus-data, but rather simply as the number of new admissions (discharges are not accounted for).  

In [11]:
hosp = pd.read_csv('https://data.cityofnewyork.us/api/views/rc75-m7u3/rows.csv?accessType=DOWNLOAD')
hosp['DATE_OF_INTEREST'] = pd.to_datetime(hosp['DATE_OF_INTEREST'])
hosp.set_index('DATE_OF_INTEREST', inplace=True)
hosp = hosp[START_DATE:]

### Most Recent New Hospital Admissions

Most recent 10 calendar days.  Note that the most recent data shows '0' hospitalizations, reflecting the lag in the data.

In [12]:
pd.DataFrame(hosp['HOSPITALIZED_COUNT'].tail(10))

,HOSPITALIZED_COUNT
DATE_OF_INTEREST,
2020-07-29,25
2020-07-30,31
2020-07-31,19
2020-08-01,22
2020-08-02,30
2020-08-03,28
2020-08-04,31
2020-08-05,10
2020-08-06,6


In [13]:
hosp['rolling_7_day_avg_hospitalization'] = hosp.rolling(7)['HOSPITALIZED_COUNT'].mean()

## Hospitalized Count, Removing the Last Two Days for Lags

In [14]:
px.line(hosp.reset_index()[:-2], 
        x='DATE_OF_INTEREST', 
        y='HOSPITALIZED_COUNT', 
        title = 'Hospitalized Count',
        template = 'simple_white')

In [15]:
px.line(hosp.reset_index()[7:-2], 
        x='DATE_OF_INTEREST', 
        y='rolling_7_day_avg_hospitalization', 
        title = 'Hospitalized Count, 7 Days Rolling Average',
        template = 'simple_white')

## Detailed Data from NYC
The analysis below pulls from the city data (which conflicts with the state data, but is more precise).  

The city uploads several CSVs every day that give **cumulative** numbers. (Why? I have no idea).  The strategy below is to grab *all* of those CSVs, concat them together, and then take the daily difference.  

Grabbing all historical versions of a file uses this link:

https://stackoverflow.com/questions/12850030/git-getting-all-previous-version-of-a-specific-file-folder

I pull this data into an AWS instance and push the raw CSVs back to github (so the world can play with the data) here: https://github.com/jlandesman/nyc_covid_data 

The data is updated via daily_update.sh.  In the future I will automate this process. 

In [22]:
## Download latest data
!git clone https://github.com/jlandesman/nyc_covid_data.git

fatal: destination path 'nyc_covid_data' already exists and is not an empty directory.


In [23]:
## Load up latest data
import os
files = os.listdir('nyc_covid_data/data/by_age')
by_age = []
for i in files: 
  tmp = pd.read_csv(os.path.join('nyc_covid_data/data/by_age', i))
  tmp['date'] = i[:10]
  by_age.append(tmp)
 
age = pd.concat(by_age)
age.head()

,AGE_GROUP,COVID_CASE_RATE,HOSPITALIZED_CASE_RATE,DEATH_RATE,date,CASE_RATE,HOSPITALIZED_RATE,CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT
0,0-17 years,279.34,23.97,0.46,2020-05-15,NaN,NaN,NaN,NaN,NaN
1,18-44 years,2042.77,222.45,18.10,2020-05-15,NaN,NaN,NaN,NaN,NaN
2,45-64 years,3332.61,819.73,167.75,2020-05-15,NaN,NaN,NaN,NaN,NaN
3,65-74 years,3342.78,1586.43,549.41,2020-05-15,NaN,NaN,NaN,NaN,NaN
4,75 and older years,4001.46,2508.34,1373.19,2020-05-15,NaN,NaN,NaN,NaN,NaN


In [24]:
## Preprocess Age data
age['date'] = pd.to_datetime(age['date'])

age['AGE_GROUP'].loc[age['AGE_GROUP'] == '0-17 years'] = '0-17'
age['AGE_GROUP'].loc[age['AGE_GROUP'] == '18-44 years'] = '18-44'
age['AGE_GROUP'].loc[age['AGE_GROUP'] == '45-64 years'] = '45-64'
age['AGE_GROUP'].loc[age['AGE_GROUP'] == '65-74 years'] = '65-74'
age['AGE_GROUP'].loc[age['AGE_GROUP'] == '75 and older years'] = '75+'

age['CASE_COUNT'].loc[age['CASE_COUNT'] == '0-17'] = 0
age['CASE_COUNT'] = age['CASE_COUNT'].astype(float)

age.sort_values('date', inplace=True)

age.set_index('date', inplace=True)
age = age['2020-06-01':]

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



## By  Age: 

Big picture as it relates to kids:  Ages 0-17 are running at ~20 cases/ day, which is roughly 7% of the total cases per day.  (As of August 6).

Based on public reporting about house parties, college students etc, it is a safe assumption that many of these cases are in the older part of the range.




In [25]:
daily_counts = age.groupby('AGE_GROUP').apply(lambda x: x['CASE_COUNT'].diff()).T.reset_index().set_index('date')
daily_counts.drop('Citywide', axis=1, inplace = True)
daily_counts = daily_counts.reset_index().melt(id_vars = 'date')
daily_counts['7_day_avg'] = daily_counts.groupby('AGE_GROUP')['value'].rolling(7).mean().reset_index()['value']
daily_counts.head()



,date,AGE_GROUP,value,7_day_avg
0,2020-06-01,0-17,NaN,NaN
1,2020-06-02,0-17,9.0,NaN
2,2020-06-03,0-17,5.0,NaN
3,2020-06-04,0-17,12.0,NaN
4,2020-06-05,0-17,26.0,NaN


# Case counts by Age

In [26]:
px.line(daily_counts.dropna(), 
        x='date', 
        y='7_day_avg',
        color = 'AGE_GROUP',
        title = 'Cases by Age Group, NYC',
        template = 'simple_white')

# Children

In [27]:
px.line(daily_counts[daily_counts['AGE_GROUP'] == '0-17'].dropna(),
        x='date',
        y='7_day_avg',
        title = 'Cases by Age Group, 0-17, NYC',
        template = 'simple_white')

# By Zip

The purpose of this section is to look at case counts in my local area in Queens.   

To do this, I take my zip code and several nearby zip codes and sum the daily cases, then take a rolling average. 

Main take away is that there are ~8 cases per day in the population nearby my child's school, in a population of ~200,000 (or ~4/ 100k people). 

If the age group 0-17 are roughly 8% of the case load, we're getting less than 1 case per day for kids in the local neighborhood.

In [29]:
import os
files = os.listdir('nyc_covid_data/data/by_zip')
by_zip = []
for i in files: 
  tmp = pd.read_csv(os.path.join('nyc_covid_data/data/by_zip', i))
  tmp['date'] = i[:10]
  by_zip.append(tmp)
 
zip = pd.concat(by_zip)
zip.head()

,MODIFIED_ZCTA,NEIGHBORHOOD_NAME,BOROUGH_GROUP,COVID_CASE_COUNT,COVID_CASE_RATE,POP_DENOMINATOR,COVID_DEATH_COUNT,COVID_DEATH_RATE,PERCENT_POSITIVE,date,TOTAL_COVID_TESTS
0,10001,Chelsea/NoMad/West Chelsea,Manhattan,366,1553.28,23563.03,21,89.12,16.10,2020-06-05,NaN
1,10002,Chinatown/Lower East Side,Manhattan,1049,1366.68,76755.41,147,191.52,21.83,2020-06-05,NaN
2,10003,East Village/Gramercy/Greenwich Village,Manhattan,447,830.83,53801.62,33,61.34,12.89,2020-06-05,NaN
3,10004,Financial District,Manhattan,31,849.17,3650.61,1,27.39,12.60,2020-06-05,NaN
4,10005,Financial District,Manhattan,61,726.53,8396.11,2,23.82,11.34,2020-06-05,NaN


In [30]:
zip['date'] = pd.to_datetime(zip['date'])
zip['COVID_CASE_COUNT'] = zip['COVID_CASE_COUNT'].astype('int')
zip['PERCENT_POSITIVE'] = zip['PERCENT_POSITIVE'].astype('float')
zip['POP_DENOMINATOR'] = zip['POP_DENOMINATOR'].astype('float')

zip.sort_values('date',inplace=True)
zip.set_index('date', inplace=True)
zip = zip['2020-06-01':]

In [31]:
new_cases = zip.groupby('MODIFIED_ZCTA').apply(lambda x: x['COVID_CASE_COUNT'].diff()).reset_index()

In [32]:
new_cases.head()

,MODIFIED_ZCTA,date,COVID_CASE_COUNT
0,10001,2020-06-01,NaN
1,10001,2020-06-02,-1.0
2,10001,2020-06-03,2.0
3,10001,2020-06-04,1.0
4,10001,2020-06-05,3.0


In [33]:
nearby_zips = [11375, 11415, 11418, 11385,11379,11367]
local_cases = pd.DataFrame(new_cases[new_cases['MODIFIED_ZCTA'].isin(nearby_zips)].groupby('date').sum()['COVID_CASE_COUNT'].rolling(7).mean()).reset_index()
local_cases.tail()

,date,COVID_CASE_COUNT
61,2020-08-02,10.43
62,2020-08-03,9.57
63,2020-08-04,9.43
64,2020-08-05,8.43
65,2020-08-06,8.29


In [38]:
px.line(local_cases.dropna(), 
        x='date', 
        y='COVID_CASE_COUNT',
        title = 'Cases per day in Zip Codes {}'.format(nearby_zips),
        template = 'simple_white')

## Total Population in Nearby Zips


In [35]:
local_population = zip[(zip['MODIFIED_ZCTA'].isin(nearby_zips)) & (zip.index == '2020-08-06')][['MODIFIED_ZCTA', 'POP_DENOMINATOR']].tail()
local_population

,MODIFIED_ZCTA,POP_DENOMINATOR
date,,
2020-08-06,11375,70552.99
2020-08-06,11418,38105.16
2020-08-06,11415,19234.25
2020-08-06,11385,102929.91
2020-08-06,11379,36195.55


In [36]:
local_population['POP_DENOMINATOR'].sum()

267017.86

## Zips with most Cases

In [37]:
rolling_30 = new_cases.set_index('date').groupby('MODIFIED_ZCTA').apply(lambda x: x['COVID_CASE_COUNT'].rolling(30).mean()).reset_index()
rolling_30 = rolling_30.groupby('date').apply(lambda x: x.sort_values('COVID_CASE_COUNT')).tail(50)

In [ ]:
rolling_30 = rolling_30.drop('date',axis=1).reset_index()
rolling_30.drop('level_1',inplace=True, axis=1)
rolling_30.tail()

,date,MODIFIED_ZCTA,COVID_CASE_COUNT
45,2020-08-06,10458,4.800000
46,2020-08-06,10467,5.533333
47,2020-08-06,11220,5.600000
48,2020-08-06,11368,5.766667
49,2020-08-06,10457,5.933333


In [ ]:
rolling_30  = rolling_30.join(zip.reset_index().loc[:,['MODIFIED_ZCTA', 'NEIGHBORHOOD_NAME','BOROUGH_GROUP']], on='MODIFIED_ZCTA', how = 'left', lsuffix = '_l')

In [ ]:
rolling_30.sort_values(['date','COVID_CASE_COUNT'], inplace=True)
px.bar(rolling_30.groupby('date').apply(lambda x: x.sort_values('COVID_CASE_COUNT')).tail(10), 
       x="COVID_CASE_COUNT", 
       y="NEIGHBORHOOD_NAME",
       color='BOROUGH_GROUP', 
       orientation='h')